# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

In [1]:
from pathlib import Path
from db_connections import client,db_neo4j,db_redis
from src import mongo,neo4j,utils,redis
from pymongo.errors import ConnectionFailure
from datetime import datetime
import os
import pandas as pd

## Conexión con bases de datos

In [2]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


## Limpieza previa

In [3]:
# --- LIMPIAR DATOS EXISTENTES EN TODAS LAS BASES ---

print("\nLimpiando datos existentes...")

# --- MONGO ---
nombre_base = "clase"
client.drop_database(nombre_base)
print("   ✓ Base de datos Mongo eliminada")

# --- REDIS ---
db_redis.flushdb()
print("   ✓ Base de datos Redis limpiada")

# --- NEO4J ---
with db_neo4j.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
print("   ✓ Nodos y relaciones de Neo4j eliminados")

print("\n Todos los datos fueron limpiados correctamente.")



Limpiando datos existentes...
   ✓ Base de datos Mongo eliminada
   ✓ Base de datos Redis limpiada
   ✓ Nodos y relaciones de Neo4j eliminados

 Todos los datos fueron limpiados correctamente.


## Carga de Datos

In [4]:
#-------------------------
# Archivos y colecciones
#-------------------------
nombre_base = "clase"
nombre_colecciones = ["usuarios", "destinos", "hoteles", "reservas", "actividades"]
nombre_archivos = ["usuarios.csv", "destinos.csv", "hoteles.csv", "reservas.csv", "actividades.csv"]

for nombre_archivo, nombre_coleccion in zip(nombre_archivos, nombre_colecciones):
    ruta = Path("fuentes") / nombre_archivo
    df = utils.lectura_csv(ruta)
    
    if df is not None and not df.empty:
        mongo.crear_coleccion(nombre_base, nombre_coleccion, recrear=True)       
        filas = df.to_dict(orient="records")
        
        # MongoDB: Usuarios, Destinos, Hoteles, Reservas, Actividades

        if nombre_coleccion == "reservas":
            df_filtrado = df[df["estado"].notna() & (df["estado"].astype(str).str.strip() != "")]
            mongo.insertar_muchos_coleccion(nombre_base, nombre_coleccion, df_filtrado.to_dict(orient="records"))
        else:
            mongo.insertar_muchos_coleccion(nombre_base, nombre_coleccion, filas)

        # Redis: usuarios conectados

        if nombre_coleccion == "usuarios":
            conectados = redis.guardar_usuarios_conectados(df, 15)
            print(f"✅ Se registran {conectados} usuarios conectados")
        
        # Neo4j: Solo para usuarios y destinos
        
        filas = df.to_dict(orient="records") #se pisa nuevamente filas porque Mongo le agrega _id y generar error para Neo4j

        if nombre_coleccion in ["usuarios", "destinos"]:
            nombre_nodo = "Usuario" if nombre_coleccion == "usuarios" else "Destino"
            campo_clave = "usuario_id" if nombre_nodo == "Usuario" else "destino_id"

            with db_neo4j.session() as session:
                for fila in filas: # Recorro  y agrego
                    session.execute_write(neo4j.crear_nodo, nombre_nodo, campo_clave, fila)
                print(f"✅ Nodos de {nombre_nodo} creados exitosamente")
        
        # Neo4j: relaciones VISITO entre Usuario y Destino

        if nombre_coleccion == "reservas":
            HOY = datetime.today().date()   
            with db_neo4j.session() as session: 
                for fila in filas: 
                    estado = str(fila.get("estado", "")).strip()
                    fecha = fila.get("fecha_reserva")

                    # Solo si el estado es Confirmada o Pagada y fecha <= hoy
                    if estado in ["Confirmada", "Pagada"] and fecha <= str(HOY):
                        session.execute_write(
                            neo4j.crear_relacion_unidireccional,
                            "Usuario", "usuario_id", fila["usuario_id"],
                            "Destino", "destino_id", fila["destino_id"],
                            "VISITO")                   
            print("✅ Relaciones VISITO creadas exitosamente")

            # Redis: reservas temporales (sin estado - en carrito)
            
            df_reservas_temporales = df[df["estado"].isna()]
            resultado = redis.carga_masiva_reservas_temporales(df_reservas_temporales)
            print(f"✅ Se insertaron {resultado} reservas temporales")

# Neo4j: Relaciones entre usuarios

ruta_relaciones = Path("fuentes") / "usuarios_relaciones.csv"
df_usuarios_relaciones = utils.lectura_csv(ruta_relaciones)

if df_usuarios_relaciones is not None and not df_usuarios_relaciones.empty:
    with db_neo4j.session() as session: 
        filas = df_usuarios_relaciones.to_dict(orient="records")
        for fila in filas:
            session.execute_write(
                neo4j.crear_relacion_bidireccional,
                "Usuario", "usuario_id", fila["usuario1"],
                "Usuario", "usuario_id", fila["usuario2"],
                fila["tipo"]
            )
        print("✅ Relaciones entre usuarios creadas exitosamente")

✅ Se insertaron 60 documentos en 'usuarios'.
✅ Se registran 15 usuarios conectados
✅ Nodos de Usuario creados exitosamente
✅ Se insertaron 37 documentos en 'destinos'.
✅ Nodos de Destino creados exitosamente
✅ Se insertaron 118 documentos en 'hoteles'.
✅ Se insertaron 1703 documentos en 'reservas'.
✅ Relaciones VISITO creadas exitosamente
✅ Se insertaron 408 reservas temporales
✅ Se insertaron 111 documentos en 'actividades'.
✅ Relaciones entre usuarios creadas exitosamente


## Verificación de la carga

### MongoDB

##### Cantidad de documentos existentes en cada colección

In [5]:
for i in range (len(nombre_colecciones)):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

La cantidad de documentos de la coleccion usuarios es 60
La cantidad de documentos de la coleccion destinos es 37
La cantidad de documentos de la coleccion hoteles es 118
La cantidad de documentos de la coleccion reservas es 1703
La cantidad de documentos de la coleccion actividades es 111


### Neo4j

In [6]:
df_nodos = neo4j.consulta(db_neo4j, """
    MATCH (n)
    RETURN labels(n)[0] AS tipo_nodo, COUNT(*) AS cantidad
""")

df_relaciones = neo4j.consulta(db_neo4j, """
    MATCH ()-[r]->()
    RETURN type(r) AS tipo_relacion, COUNT(*) AS cantidad
""")

print("Nodos por tipo:")
print(df_nodos.to_string(index=False))

print("\nRelaciones por tipo:")
print(df_relaciones.to_string(index=False))


Nodos por tipo:
tipo_nodo  cantidad
  Usuario        60
  Destino        37

Relaciones por tipo:
tipo_relacion  cantidad
       VISITO       662
  FAMILIAR_DE       120
     AMIGO_DE       470


### Redis

##### Cantidad y Vista Previa de Usuarios Conectados

In [7]:
claves = db_redis.keys("usuario:*:sesion")
print(f"Cantidad de usuarios conectados {len(claves)}\n")
print("Se imprimen los primeros 5:")
for clave in claves[:5]:
    usuario_id = clave.split(":")[1]
    estado = db_redis.get(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"Usuario {usuario_id} → sesión: {estado} | TTL: {tiempo_restante} segundos")

Cantidad de usuarios conectados 15

Se imprimen los primeros 5:
Usuario 51 → sesión: activa | TTL: 3592 segundos
Usuario 1 → sesión: activa | TTL: 3592 segundos
Usuario 4 → sesión: activa | TTL: 3592 segundos
Usuario 14 → sesión: activa | TTL: 3592 segundos
Usuario 49 → sesión: activa | TTL: 3592 segundos


##### Cantidad y Vista Previa de Reservas en Carrito

In [8]:
claves = db_redis.keys("reserva_temp:*")
print(f"Cantidad de reservas temporales {len(claves)}\n")
print("Se imprimen las primeras 5:")
for clave in claves[:5]:
    datos = db_redis.hgetall(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"{clave}: {datos} | TTL: {tiempo_restante} segundos")

Cantidad de reservas temporales 408

Se imprimen las primeras 5:
reserva_temp:1375: {'usuario_id': '10', 'destino_id': '24', 'fecha_reserva': '2025-03-24', 'precio_total': '79230'} | TTL: 3594 segundos
reserva_temp:143: {'usuario_id': '4', 'destino_id': '4', 'fecha_reserva': '2024-12-31', 'precio_total': '201366'} | TTL: 3594 segundos
reserva_temp:155: {'usuario_id': '2', 'destino_id': '4', 'fecha_reserva': '2024-10-29', 'precio_total': '191727'} | TTL: 3594 segundos
reserva_temp:921: {'usuario_id': '48', 'destino_id': '16', 'fecha_reserva': '2025-01-24', 'precio_total': '88945'} | TTL: 3594 segundos
reserva_temp:2078: {'usuario_id': '9', 'destino_id': '36', 'fecha_reserva': '2025-06-03', 'precio_total': '94068'} | TTL: 3594 segundos


## Generamos algunas búsquedas en caché

In [10]:
#Borrar redis
#db_redis.flushdb()


consultas = [
    ("hoteles", {"ciudad": "La Plata"}),
    ("destinos", {"tipo": "Playa", "ciudad": "La Rioja"}),
    ("hoteles", {"nombre": "Familia Galiano S.L. Hotel"}),
    ("actividades", {"tipo": "relax"})
]

for tipo, filtro in consultas:
    resultado = redis.obtener_cache(tipo, filtro)
    if resultado is None:
        cursor = mongo.obtener_cursor("clase",nombre_coleccion=tipo,filtro=filtro,proyeccion={"_id":0})
        resultado = list(cursor)
        if resultado:
            redis.guardar_en_cache(tipo,filtro,resultado,ttl=10)   

print("✅ Caché de búsqueda creado")

✅ Caché de búsqueda creado


### Vista del caché de búsqueda para lo guardado

In [12]:
for tipo, filtro in consultas:
    resultados = redis.obtener_cache(tipo, filtro)
    if resultados:
        for resultado in resultados:
            print(resultado)
    else:
        print(f"No existe información en caché para la búsqueda {filtro} en {tipo}")

No existe información en caché para la búsqueda {'ciudad': 'La Plata'} en hoteles
No existe información en caché para la búsqueda {'tipo': 'Playa', 'ciudad': 'La Rioja'} en destinos
No existe información en caché para la búsqueda {'nombre': 'Familia Galiano S.L. Hotel'} en hoteles
No existe información en caché para la búsqueda {'tipo': 'relax'} en actividades
